In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections as co

data = pd.read_csv('C:/Users/asus-pc/Desktop/kdd/homework2/datacsv.csv')

def stats_str(df):
    result=dict(co.Counter(df))
    return result

#数据预处理
df = pd.DataFrame({"Permit Type":data['Permit Type'], 
 "Permit Type Definition":data['Permit Type Definition'],
  "Current Status":data['Current Status']},
  columns =['Permit Type','Permit Type Definition','Current Status'])
data2=df.values[0::1000,:]
dataindex=list(pd.Series(stats_str(data2[:,0])).index)+list(pd.Series(stats_str(data2[:,1])).index)+list(pd.Series(stats_str(data2[:,2])).index)
for i in range(len(dataindex)):
    data2[data2==dataindex[i]]=i

#关联性分析 
def createC1(dataSet):                  
    C1 = []    
    for transaction in dataSet:    
        for item in transaction:    
            if not [item] in C1:    
                C1.append([item])
    C1.sort()    
    return map(frozenset, C1)

def scanD(D,Ck,minSupport):             
    ssCnt = {}    
    for tid in D:                        
        for can in Ck:                      
            if can.issubset(tid):          
                if not can in ssCnt:
                    ssCnt[can] = 1    
                else:    
                    ssCnt[can] +=1    
    numItems = float(len(D))      
    retList  = []    
    supportData = {}    
    for key in ssCnt:    
        support = ssCnt[key]/numItems   
        if support >= minSupport:    
            retList.insert(0,key)       
        supportData[key] = support    
    return retList, supportData    
    
def aprioriGen(Lk, k):                  
    retList = []    
    lenLk   = len(Lk)    
    for i in range(lenLk):    
        for j in range(i+1, lenLk):     
            L1 = list(Lk[i])[:k-2]    
            L2 = list(Lk[j])[:k-2]    
            L1.sort()    
            L2.sort()    
            if L1==L2:    
                retList.append(Lk[i]|Lk[j])    
    return retList    
    
def apriori(dataSet, minSupport = 0.5):    
    C1 = list(createC1(dataSet))    
    D  = list(map(set,dataSet))    
    L1, supportData = scanD(D,C1,minSupport)    
    L  = [L1]                              
    k  = 2    
    while (len(L[k-2])>0):                  
        Ck = aprioriGen(L[k-2], k)    
        Lk, supK = scanD(D, Ck, minSupport)    
        supportData.update(supK)        
        L.append(Lk)    
        k +=1    
    return L,supportData 

L, suppData = apriori(data2)